In [ ]:
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
import torch

import sys, os
sys.path.append(os.path.abspath('..'))
from models.subpix_rcnn import SubpixRCNN
from utils import move_data_to_device, move_dict_to_cpu, plot_image_boxes, evaluate_predictions
from PsfSimulator import PsfDataset
from scripts.plotting import PlotController
from scripts.evaluating import test_densities_fixed_snr, test_model_once_plot, evaluate_recall_per_snr, test_perlinnoise_fixed_snr
import numpy as np
import matplotlib.pyplot as plt
import pickle

from scripts.test_gpu_tracking import load_csv, extract_predictions_from_gpu_tracking, dist_evaluate_prediction, dist_evaluate_predictions

In [ ]:
seed = None
num_datapoints = 1
num_spots_min = 1
num_spots_max = 1
sigma_mean= 1.0
sigma_std = 0.1
snr_min = 5
snr_max = 5
snr_std = 0.0
base_noise_min = 50
base_noise_max = 6000
use_gauss_noise = False
gauss_noise_std = 0.02
use_perlin_noise = False
perlin_min_max = (0.4, 0.6)
img_w = 64
img_h = 64

dataset = PsfDataset(seed, num_datapoints, num_spots_min, num_spots_max, sigma_mean, sigma_std,
                      snr_min, snr_max, snr_std, base_noise_min, base_noise_max, use_gauss_noise,
                      gauss_noise_std, use_perlin_noise, perlin_min_max, img_w, img_h)



In [ ]:
# Load model
device = 'cuda'
backbone = resnet_fpn_backbone("resnet50", pretrained=False, trainable_layers=5)
kwargs = {"nms_thresh": 0.1, "detections_per_img": None, "score_thresh": 0.7, "image_mean":[0,0,0], "image_std":[1,1,1]}
model = SubpixRCNN(backbone, num_classes=2, device=device, **kwargs)
model.to(device=device)
path = r"D:\zeiss\Desktop\coding\Hilger\bachelor\notebooks\subpix_rcnn_models\2025-05-28_08-57-04\threehrsperlin.pth"
model.load_state_dict(torch.load(path, map_location=device))
model.eval()

In [ ]:
# Load GPU tracking results